# 正态性检验方法
**正态性检验（Normality Test）**是统计学中用于判断数据是否服从正态分布（高斯分布）的重要方法。正态分布是许多统计模型（如t检验、ANOVA、线性回归等）的基础假设，因此验证数据的正态性至关重要。

正态性检验方法主要有**Shapiro-Wilk检验**、Kolmogorov-Smirnov检验 (K-S检验)、Anderson-Darling检验

| 检验方法               | Shapiro-Wilk检验                          | Kolmogorov-Smirnov检验 (K-S检验)           | Anderson-Darling检验                     |
|------------------------|------------------------------------------|------------------------------------------|------------------------------------------|
| **适用场景**           | 小样本（n ≤ 50）效果更好                  | 大样本，或已知参考分布时                  | 中小样本，尤其关注尾部拟合               |
| **原假设 (H₀)**        | 数据来自正态分布                          | 数据与指定分布（如正态分布）一致           | 数据来自特定分布（如正态分布）            |
| **统计量特点**         | 基于样本数据的顺序统计量                  | 基于经验分布函数与理论分布的最大差异       | 加权平方差异（对尾部更敏感）              |
| **优点**               | 小样本功效高                              | 适用于任何分布比较，无需参数估计           | 对尾部异常敏感，适合极端值检测           |
| **缺点**               | 大样本时可能过于敏感（易拒绝）             | 对分布参数未知时功效较低（需Lilliefors修正） | 计算较复杂                               |
| **参数估计需求**       | 不需要预先估计分布参数                     | 若参数未知需估计（μ,σ），需用Lilliefors修正 | 通常需要估计分布参数                      |
| **适用数据类型**       | 连续数据                                  | 连续或离散数据                            | 连续数据                                 |
| **样本量限制**         | 3 ≤ n ≤ 5000（常见实现）                  | 无严格限制，但大样本更稳定                | 无严格限制                               |

# 数值法（统计检验法）

## Anderson-Darling检验

### 基本原理
**Anderson-Darling（A-D）**检验是一种非参数检验，用于评估样本数据是否来自某个特定分布（如正态分布、指数分布等）。其核心思想是：
- 通过比较样本经验分布函数（ECDF）与理论分布函数（CDF）的加权差异，重点捕捉分布尾部的偏离。
- 相比K-S检验，A-D检验对分布的尾部差异更敏感，尤其适合检测极端值的影响。

### 适用场景
- 适用于**中小样本**（n < 1000），尤其是金融、医学、工程等领域对正态性要求严格的场景。
- **对尾部偏离敏感**：能有效检测数据在分布两端（极端值）的偏离，比Shapiro-Wilk和K-S检验更严格。
- **示例**：
    - 检验股票收益率是否服从正态分布（极端事件风险分析）。
    - 医学数据中异常值的正态性检测（如药物剂量反应）。

### 零假设与备择假设
- 零假设（$H_0$）：样本数据来自指定的理论分布
- 备择假设（$H_1$）：样本数据不服从该分布

### 检验统计量
$$A^2 = -n - \frac{1}{n} \sum_{i=1}^n (2i-1) \left[ \ln F(X_i) + \ln (1 - F(X_{n+1-i})) \right]$$
其中：
- $F(X_i)$是理论分布的累积分布函数（CDF）。
- $n$为样本量。
- 统计量$A^2$越大，样本与理论分布的偏离越显著。

### 检验步骤
以正态性检验为例
- **参数估计**（若分布参数未知）： 
    - 样本均值$\hat{\mu} = \bar{X}$，标准差$\hat{\sigma} = s$。
- **计算理论CDF**： 
    - 对每个$X_i$，计算$F(X_i) = \Phi\left(\frac{X_i - \hat{\mu}}{\hat{\sigma}}\right)$。
- **计算统计量$A^2$与p值**： 
    - **排序数据**：将样本$X_1$, $X_2$, $\ldots$, $X_n$ 按升序排列。
    - **计算理论CDF**：对每个$X_i$，计算$F(X_i)$（如正态分布$\Phi\left(\frac{X_i - \mu}{\sigma}\right)$）。
    - **计算统计量**： 
        - $$A^2 = -n - \frac{1}{n} \sum_{i=1}^n (2i-1) \left[ \ln F(X_i) + \ln (1 - F(X_{n+1-i})) \right]$$
    - **调整统计量**：根据分布类型修正$A^2$（如正态分布需乘以$(1 + \frac{4}{n} - \frac{25}{n^2})$）。
    - **p值**：需查A-D检验专用表，或通过软件计算p值。
- **p值判断**： 
    - $p < α$时拒绝原假设。

### A-D检验实际案例

#### 案例背景
某视频平台测试新版推荐算法对用户观看时长的影响：
- **A组（对照组）**：1000名用户使用旧版算法（基于热门视频推荐）
- **B组（实验组）**：1000名用户使用新版算法（基于用户兴趣画像）
- **分析需求**：验证B组用户的观看时长是否服从正态分布（决定后续使用检验方法）。

In [5]:
import numpy as np
from scipy import stats
from scipy.stats import norm

# 设置随机种子保证可复现
np.random.seed(42)

# 生成B组用户浏览时长数据
watch_time = np.abs(norm.rvs(loc=60, scale=30, size=1000, random_state=0))  # 绝对值保证非负
watch_time = np.round(watch_time, 1)

print("B组观看时长统计:")
print(f"均值 = {np.mean(watch_time):.1f} 秒")
print(f"标准差 = {np.std(watch_time):.1f} 秒")

B组观看时长统计:
均值 = 59.2 秒
标准差 = 28.5 秒


#### 确定假设
- 零假设（$H_0$）：样本数据服从正态分布
- 备择假设（$H_1$）：样本数据不服从正态分布

#### `stats.anderson()`
- 输入参数
   - `x`: 要进行检验的数据样本，必须是一维，样本量建议不低于 8 个
   - `dist`: 指定检验的目标理论分布
       - `norm`：正态分布
       - `expon`：指数分布
- 返回值
    - `statistic`: Anderson-Darling 检验统计量 $A^2$。越大表示与目标分布差异越大
    - `critical_values`: 各个显著性水平（15%、10%、5%、2.5%、1%）下的临界值数组
    - `significance_level`: 与临界值一一对应的显著性水平数组（百分比）

#### `stats.anderson()`代码实现

In [8]:
# 执行A-D检验（使用样本均值和标准差）
result = stats.anderson(watch_time, dist='norm')

# 输出结果
print(f"\nA-D统计量: {result.statistic:.3f}")
print("临界值表:")
for cv, sl in zip(result.critical_values, result.significance_level):
    print(f"{sl:.1f}% 置信水平: {cv:.3f}")

# 判断结果（95%置信水平）
if result.statistic > result.critical_values[2]:  # 对应α=0.05
    print("\n结论：拒绝H₀，数据不服从正态分布（p < 0.05）")
else:
    print("\n结论：无法拒绝H₀，数据可能服从正态分布")


A-D统计量: 1.063
临界值表:
15.0% 置信水平: 0.574
10.0% 置信水平: 0.653
5.0% 置信水平: 0.784
2.5% 置信水平: 0.914
1.0% 置信水平: 1.088

结论：拒绝H₀，数据不服从正态分布（p < 0.05）


In [9]:
result

AndersonResult(statistic=1.0625869415803209, critical_values=array([0.574, 0.653, 0.784, 0.914, 1.088]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=59.194, scale=28.5070281624489)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')